In [263]:
from typing import List, Dict
import matplotlib.pyplot as plt
import random
import copy
import numpy as np
import queue
from collections import deque

class Elf():
    def __init__(self, x,y):
        self.x = x
        self.y=y
        self.active = True
        self.last_move =""


    def move(self, direction):
        self.active = True
        if direction == 'n':
            self.x -= 1
        elif direction == 's':
            self.x += 1
        elif direction == "w":
            self.y -= 1
        else:
            self.y += 1

    def step(self,adjacent_positions, action_queue):
        values = adjacent_positions.values()
        if  sum(values) ==0:
            self.active = False

            return # no elf in adjancent positions, do nothing

        self.prev_position = (self.x, self.y) #backup


        for next_action in action_queue:
            considered_positions = [adjacent_positions[x] for x in adjacent_positions.keys() if next_action in x]
            if sum(considered_positions) ==0:
                self.move(next_action)
                
                break
        
        
    def reverse(self):
        self.x, self.y = self.prev_position


class Environment:
    def __init__(self, file_path="day23/input.txt"):
        with open(file_path,"r") as file:
            lines = file.readlines()
        raw_data = [line.strip() for line in lines]
        raw_temp =[]
        self.elfs = []
        for line in raw_data:
            temp = []
            for element in line:
                temp.append(element)
            raw_temp.append(temp)

        raw_data = np.array(raw_temp)
        data = np.where(raw_data=="#", 1, 0 )
        data = np.pad(data, 1000, constant_values=0)
        self.data = data
        i=0
        for line in data:
            j =0
            for element in line:
                if element == 1:
                    elf = Elf(i,j) #initialize elf from data
                    self.elfs.append(elf)
                j += 1
            i += 1
        self.time =0
        self.game_status = "active"
        self.initial= True
        self.action_queue = deque(['n', 's','w', 'e']) 

    def update_matrix(self):
        new_data = np.zeros((self.data.shape[0], self.data.shape[1]))
        for elf in self.elfs:
            new_data[elf.x, elf.y] += 1
        self.data = new_data
    def get_adjacent_cells(self, x,y): #given x,y, get adjacent cell
        #north: x-1, y
        n,ne,nw, s, se, sw, w, e = 2, 2, 2, 2,2,2,2,2 
        if x - 1 >=0: 
            n = self.data[x-1, y]
        if x -1 >=0 and y-1 >=0:
            nw = self.data[x-1, y-1]
        if x-1 >=0 and y +1 <= self.data.shape[1]-1:
            ne = self.data[x-1, y+1]
        if x+1 <= self.data.shape[0] -1:
            s = self.data[x+1, y]
        if x+1 <= self.data.shape[0] -1 and y +1 <= self.data.shape[1]-1:
            se = self.data[x+1, y+1]
        if x+1 <= self.data.shape[0] -1 and y -1 >= 0:
            sw = self.data[x+1, y-1]
        if  y -1 >= 0:
            w = self.data[x, y-1]
        if y +1 <= self.data.shape[1]-1:
            e = self.data[x, y+1]
        return {"n":n,"ne":ne,"nw":nw,"s":s, "se":se,"sw":sw,"w":w,"e":e}


    def check_status(self):
        active = False
        for elf in self.elfs:
            if elf.active:
                active = True
                break
        return active
        
    def validate(self):
        data = np.array(self.data)
        return list(np.argwhere(data>1))
    def step(self):
        self.initial= False
        for elf in self.elfs:
            adjacent_positions = self.get_adjacent_cells(elf.x, elf.y)
            elf.step(adjacent_positions, self.action_queue)
        self.update_matrix()
        invalid_cells = self.validate()
        if len(invalid_cells)>0:
            for cell in invalid_cells:
                for elf in self.elfs:
                    if (cell[0], cell[1]) == (elf.x, elf.y):
                        elf.reverse()
            self.update_matrix()

        self.action_queue.rotate(-1)



    def printout(self):
        i=0
        for row in self.data:
            j=0
            print()
            for item in row:
                if item!=0:
                    print("#", end="")
                else:
                    print(".", end="")
                j += 1
            i += 1

    def count_empty_ground_tiles(self):
        x =[]
        y = []
        for elf in self.elfs:
            x.append(elf.x)
            y.append(elf.y)

        min_x = min(x)
        max_x = max(x)
        min_y = min(y)
        max_y = max(y)
        min_rectagle = np.zeros((max_x- min_x+1,max_y - min_y+1))
        for elf in self.elfs:
            min_rectagle[elf.x-min_x, elf.y-min_y] = 1
        return sum(min_rectagle.flatten()==0)


    def get_n_rounds(self):
        n =0 
        while self.check_status():
            self.step()
            n += 1
        return n


In [264]:
env = Environment(file_path="day23/input.txt")
# for i in range(12):
#     env.step()
# env.printout() .

# env.count_empty_ground_tiles()
env.get_n_rounds()

916

In [ ]:
from typing import List
import matplotlib.pyplot as plt
import random
import copy
class Position:
    def __init__(self, x,y):
        self.x = x
        self.y=y
class Blizzard(Position):
    def __init__(self, x, y,direction):
        self.direction = direction
        super().__init__(x, y)
    def step(self):
        if self.direction =="up":
            self.y += 1
        if self.direction =="down":
            self.y += -1
        if self.direction =="left":
            self.x += -1
        if self.direction =="right":
            self.x += 1
class Elf(Position):
    def step(self,direction:str):
        if direction =="up":
            self.y += 1
        elif direction =="down":
            self.y += -1
        elif direction =="left":
            self.x += -1
        elif direction =="right":
            self.x += 1

class Environment:
    def __init__(self, blizzards:List[Blizzard], walls: List[Position], entrance: Position,elf:Elf, data):
        self.data = data
        self.blizzards = blizzards
        self.elf = elf
        self.walls = walls
        self.max_x = max([wall.x for wall in self.walls])
        self.min_x = min([wall.x for wall in self.walls])
        self.max_y = max([wall.y for wall in self.walls])
        self.min_y = min([wall.y for wall in self.walls])
        self.entrance = entrance
        self.time=0
        self.game_status = "active"
        self.init=True
    def check_blizzard_against_wall(self, blizzard):
        if blizzard.x == self.max_x or blizzard.x == self.min_x or \
           blizzard.y == self.max_y or blizzard.y == self.min_y: #if the blizzard reaches a wall
            for wall in self.walls: 
                if blizzard.x == wall.x and blizzard.y == wall.y: #identify the wall
                    #move the blizzard to the oposite direction
                        #first, find the opposite wall cell
                        for target_wall in self.walls:
                            if  (wall.x == target_wall.x and wall.y != target_wall.y):
                                # return the new position for the blizzard
                                if blizzard.direction == "up":
                                    return (target_wall.x, target_wall.y+1)
                                elif (blizzard.direction == "down"):
                                    return (target_wall.x, target_wall.y-1)
                            if (wall.y == target_wall.y and wall.x != target_wall.x):
                                if blizzard.direction == "left":
                                    return (target_wall.x-1, target_wall.y)
                                if blizzard.direction == "right":
                                    return (target_wall.x+1, target_wall.y)
        return None
    def square_distance(self):
        return (self.elf.x- self.entrance.x)**2+(self.elf.y- self.entrance.y)**2

    def fastforward(self, n_step:int):
        def move_blizzard(n_step: int, direction: str, current_position: int, field_length: int):
            #calculate the new position after moving n_step
            if direction == "right" or direction == "up": #same direction with index
                mod= (n_step + current_position)%(field_length-1)
                if mod ==0:
                    return field_length-1
                else:
                    return mod
            else:
                return field_length-1 - abs((current_position -n_step)%(field_length-1))
        for blizzard in self.blizzards:
            
            if blizzard.direction == "up" or blizzard.direction == "down": #vertical blizzards
                blizzard.y = move_blizzard(n_step,blizzard.direction, blizzard.y, self.max_y-self.min_y)
            else:
                blizzard.x = move_blizzard(n_step,blizzard.direction, blizzard.x, self.max_x-self.min_x)




        
            
    def step(self,direction) -> int:
        # print("in the step ", direction)
        if self.init is True and direction != "wait":
            self.init= False
        #move elf
        if self.game_status != "active":
            raise Exception ("Game is over or done")
        self.time +=1
        self.elf.step(direction)
        #blizzards moves
        for blizzard in self.blizzards:
            blizzard.step()
            result= self.check_blizzard_against_wall(blizzard)
            if result: #move the blizzard to the correct position
                blizzard.x,blizzard.y = result[0],result[1]
        if self.check_status() <0:
            self.game_status = "over"
            return -1
        elif self.check_status() >0:
            self.game_status = "done"
            return 1
        else: 
            return 0

    def check_status(self):
        if self.init: return 0
        for blizzard in self.blizzards: #hit a blizzard
            if self.elf.x == blizzard.x and self.elf.y == blizzard.y:
                return -2
        if self.elf.x == self.entrance.x and self.elf.y == self.entrance.y:
            return 1
        if self.elf.x >= self.max_x or self.elf.x <= self.min_x or \
           self.elf.y >= self.max_y or self.elf.y <= self.min_y: #if elf hits a wall
                return -1

        return 0
    def visualize(self):
        walls = [[wall.x,wall.y] for wall in self.walls ]
        entrance = [[self.entrance.x,self.entrance.y]]
        # print(entrance)
        elf = [[self.elf.x,self.elf.y]]
        plt.figure(figsize=(30,30))
        plt.plot(*zip(*walls), marker='*', color='r', ls='')
        plt.plot(*zip(*entrance), marker='+', color='b', ls='')
        plt.plot(*zip(*elf), marker='x', color='b', ls='')
        for blizzard in self.blizzards:
            if blizzard.direction == "up":
                blizzards = [[blizzard.x,blizzard.y]]
                plt.plot(*zip(*blizzards), marker='^', color='r', ls='')
            elif blizzard.direction == "down":
                blizzards = [[blizzard.x,blizzard.y]]
                plt.plot(*zip(*blizzards), marker='v', color='r', ls='')
            elif blizzard.direction == "right":
                blizzards = [[blizzard.x,blizzard.y]]
                plt.plot(*zip(*blizzards), marker='>', color='r', ls='')
            elif blizzard.direction == "left":
                blizzards = [[blizzard.x,blizzard.y]]
                plt.plot(*zip(*blizzards), marker='<', color='r', ls='')
    def clone(self):
        env = copy.deepcopy(self)
        return env
class oneStepAhead:
    def __init__(self, env:Environment):
        self.env = env
    def next_action(self):
        ok_actions=[]
        for action in ["up","down", "left", "right", "wait"]:
            env = self.env.clone()
            reward = env.step(action)
            if reward > 0: return action
            elif reward ==0:
                distance =env.square_distance()
                ok_actions.append((action,distance))
        if len(ok_actions) >0:
            # ok_actions.sort(key=lambda x: x[1])
            # return ok_actions[0][0]
            return random.choice(ok_actions)[0]
        else:
            return None

class nStepsAhead:
    def __init__(self, env:Environment):
        self.env = env
    def next_action(self):
        ok_actions=[]
        for action in ["up","down", "left", "right", "wait"]:
            env = self.env.clone()
            reward = env.step(action)
            if reward > 0: return action
            elif reward ==0:
                distance =env.square_distance()
                ok_actions.append((action,distance))
        if len(ok_actions) >0:
            # ok_actions.sort(key=lambda x: x[1])
            # return ok_actions[0][0]
            return random.choice(ok_actions)[0]
        else:
            return None

        


In [ ]:
import random
num_episodes = 1000000

best_actions =[]
steps = []
champion = 300
for game in range(num_episodes):
    actions =[]

    env = Environment()
    while env.game_status == "active" and env.time < champion:
        agent = oneStepAhead(env)
        direction = agent.next_action()
        # print(direction)
        if direction is not None:
            actions.append(direction)
            env.step(direction)
            # print("env status ", env.game_status)
        else:
            
            break
    
    if env.game_status == "done":
        champion = env.time
        best_actions= actions
        print("got new champion finishing in ", env.time)
    # else:
    #     if len(actions) >1:
    #         print(actions)
print("best actions ", best_actions)




In [2]:
from typing import List
import matplotlib.pyplot as plt
import random
import copy
import numpy as np
class Position:
    def __init__(self, x,y):
        self.x = x
        self.y=y
class Elf(Position):
    def step(self,direction:str):
        self.prev = (self.x, self.y)
        if direction =="up":
            self.x += -1
        elif direction =="down":
            self.x += 1
        elif direction =="left":
            self.y += -1
        elif direction =="right":
            self.y += 1
    def backup(self):
        self.x, self.y = self.prev
class Environment:
    def __init__(self, entrance=None,file_path="day24/input.txt", elf =None):
        with open(file_path,"r") as file:
            lines = file.readlines()
        raw_data = [line.strip() for line in lines]
        raw_temp =[]
        for line in raw_data:
            temp = []
            for element in line:
                temp.append(element)
            raw_temp.append(temp)
        raw_data = np.array(raw_temp)
        inner_data = raw_data[1:raw_data.shape[0]-1,1:raw_data.shape[1]-1]
        outer_data = np.where(raw_data=="#", 1, 0 )
        if not elf:
            self.elf = Elf(0, 1)
        else:
            self.elf = Elf(elf[0], elf[1])
        outer_data[self.elf.x,self.elf.y] = 0

        if not entrance:
            self.entrance = outer_data.shape[0]-1,outer_data.shape[1]-2

        else:
            self.entrance = entrance
        outer_data[self.entrance[0],self.entrance[1]] =-1

        self.outer_data = outer_data
        self.left_data = np.where(inner_data=="<", 2, 0 )
        self.right_data = np.where(inner_data==">",4, 0)
        self.up_data = np.where(inner_data=="^",3, 0)
        self.down_data = np.where(inner_data=="v",5, 0)
        self.time =0
        self.game_status = "active"
        self.initial= True
    def check_status(self):
        data = self.get_data()
        if (self.elf.x, self.elf.y) == self.entrance:
            return 1
        if not self.initial and (self.elf.x >= data.shape[0]-1 or self.elf.y >= data.shape[1]-1 or self.elf.x <0 or self.elf.y <0):
            return -1
        value = data[self.elf.x][self.elf.y]

        if value == -1:
            return 1
        elif value ==0:
            return 0
        else: return -1
        
    def get_data(self):
        re_inner_data = self.left_data+self.right_data+self.up_data + self.down_data
        re_inner_data = np.pad(re_inner_data,1,constant_values=0)
        return re_inner_data + self.outer_data
    def progress_env(self, n_steps, reset_state=False):
        # print("progress received n_steps ", n_steps, "reset_state ", reset_state, " time ", self.time )


        if not reset_state:
            self.time += n_steps
            roll_steps = n_steps
        else:
            roll_steps = n_steps- self.time
            self.time = n_steps
        self.left_data =np.roll(self.left_data,-roll_steps,axis =1)
        self.right_data =np.roll(self.right_data,roll_steps,axis =1)
        self.up_data =np.roll(self.up_data,-roll_steps,axis =0)
        self.down_data =np.roll(self.down_data,roll_steps,axis =0)
        # assert self.check_status() >=0, str(self.elf.x) + " " + str(self.elf.y) + " " + str(hash(str(env.get_data()))) + " time " + str(self.time)

    def backup(self):

        self.elf.backup()
        # print("call progress_env at backup")
        self.progress_env(-1)

    def step(self,direction) -> int:
        self.initial= False
        # print("call progress at step")
        self.progress_env(1)
        self.elf.step(direction)
        if self.check_status() <0:
            self.game_status = "over"
            return -1
        elif self.check_status() >0:
            self.game_status = "done"
            return 1
        else: 
            self.game_status = "active"
            return 0
    def printout(self):
        data = self.get_data()
        i=0
        for row in data:
            j=0
            print()
            for item in row:
                if (i,j) == (self.elf.x,self.elf.y):
                    if item!=0:
                        print("X", end="")
                    else:
                        print("E", end="")
                else:
                    if item==1:
                        print('#', end="")
                    elif item==0 or item ==-1:
                        print(".", end="")
                    elif item == 2:
                        print("<", end="")
                    elif item == 4:
                        print(">", end="")
                    elif item == 3:
                        print("^", end="")
                    elif item == 5:
                        print("v", end="")
                    else:
                        print("&", end="")
                j += 1
            i += 1

    def clone(self):
        env = copy.deepcopy(self)
        return env


In [3]:
import queue

# Breadth First Search #
# visited_set
# parent_map: {node: (parent, actions)}
# Queue q
# starting position
# v = starting position
# q.enqueue(starting_position)
# while q not empty():
# v = q.dequeue()
# if v is the goal then return v
# adjancent_nodes, actions = get_adjancent_nodes(v)
# for each node in adjancent_nodes:
    # if node is not in visited_set:
        #q.enqueue(node)
        # node.parent = v
        # node.parent_actions = actions
        # visited_set.add(node)
def test_action(action, env):
        reward = env.step(action)
        status = env.check_status()
        x,y, time = env.elf.x, env.elf.y, env.time
        env_state = hash(str(env.get_data()))
        env.backup()

        return reward >= 0, (x,y,env_state, time)
def get_adjacents(node, env):
    
    output=[]
    ok_actions =[]
    env_hash = hash(str(env.get_data()))
    node_id = (node[0], node[1], env_hash)
    for action in ["wait", "up","down", "left", "right"]:
        result = test_action(action, env)
        if result[0]:
            output.append((result[1][0],result[1][1],result[1][2],result[1][3],[action]))
            ok_actions.append(action)
    waits=[]
    wait_count =0
    env.progress_env(-wait_count) #reverse environment back

    return output
def find_shortest_path(file_path="day24/input.txt",starting_time=0, starting_position=(0,1), entrance = (21,150)):
    dead_nodes = []
    visited_nodes = set()

    q = queue.Queue()


    env = Environment(file_path=file_path,entrance=entrance, elf=starting_position)
    env.progress_env(starting_time, reset_state=True)

    data = env.get_data()
    entrance=(data.shape[0]-1, data.shape[1]-2 )


    parent_map = {}
    env_hash = hash(str(env.get_data()))
    starting_node = ((starting_position[0], starting_position[1], env_hash),starting_time)
    q.put(starting_node)
    solutions = []
    while not q.empty():
        node = q.get()
        # print(node)
        if (node[0][0], node[0][1]) == entrance:
            ##handle retrieving
            actions = []
            node_id = (node[0][0], node[0][1], node[0][2])
            while True:
                parent = parent_map.get(node_id,"")
                if parent !="":
                    node_id = parent[0]
                    actions = parent[1] + actions
                    if node_id == (starting_position[0], starting_position[1], env_hash):
                        break
            solutions.append(actions)
        

        
        env.elf.x, env.elf.y = node[0][0],node[0][1] #update elf position for environment
        time = node[1]
        env.progress_env(time,reset_state=True) #update the environment progress
        assert env.check_status() >= 0, "time " + str(time)  + " x, y " + str(env.elf.x) + " " + str(env.elf.y)
        adjacent_items = get_adjacents(node, env)

        for adjacent_item in adjacent_items:
            adjacent_node = (adjacent_item[0], adjacent_item[1],adjacent_item[2])
            if adjacent_node not in visited_nodes:
                visited_nodes.add(adjacent_node)
                q.put((adjacent_node,adjacent_item[3]))
                parent_map[adjacent_node]= (node[0],adjacent_item[4])
    if len(solutions)>0:
        solutions.sort(key = lambda x: len(x))
        # print("actions ", solutions[0])
        print("time ", len(solutions[0]))
        return len(solutions[0])
    else:
        print("solution not found")
stage1 = find_shortest_path(starting_time=0, starting_position=(0,1), entrance = (21,150))
print("stage 1 takes ",stage1)


time  332
stage 1 takes  332


In [60]:
stage2 = find_shortest_path(starting_time=stage1, starting_position=(21,150), entrance = (0,1))
print("stage 2 takes ",stage2)


In [ ]:
stage3 = find_shortest_path(starting_time=stage1+stage2, starting_position=(0,1), entrance = (21,150))
print("stage 3 takes ",stage3)
print("total time for 3 trips ", stage1+stage2+stage3)

Depth First Search implementation

In [197]:
env = Environment()
data = env.get_data()

In [160]:
# env = Environment(file_path="day24/example_input.txt")
# data = env.get_data()
# env.printout()


#E######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#

In [45]:
from typing import List
import matplotlib.pyplot as plt
import random
import copy
import numpy as np
class Position:
    def __init__(self, x,y):
        self.x = x
        self.y=y
class Elf(Position):
    def step(self,direction:str):
        self.prev = (self.x, self.y)
        if direction =="up":
            self.x += -1
        elif direction =="down":
            self.x += 1
        elif direction =="left":
            self.y += -1
        elif direction =="right":
            self.y += 1
    def backup(self):
        self.x, self.y = self.prev
class Environment:
    def __init__(self, entrance=None,file_path="day24/input.txt", elf =None):
        with open(file_path,"r") as file:
            lines = file.readlines()
        raw_data = [line.strip() for line in lines]
        raw_temp =[]
        for line in raw_data:
            temp = []
            for element in line:
                temp.append(element)
            raw_temp.append(temp)
        raw_data = np.array(raw_temp)
        inner_data = raw_data[1:raw_data.shape[0]-1,1:raw_data.shape[1]-1]
        outer_data = np.where(raw_data=="#", 1, 0 )
        if not elf:
            self.elf = Elf(0, 1)
            outer_data[0,1] = 0

        if not entrance:
            self.entrance = outer_data.shape[0]-1,outer_data.shape[1]-2

        else:
            self.entrance = entrance
        outer_data[self.entrance[0],self.entrance[1]] =-1

        self.outer_data = outer_data
        self.left_data = np.where(inner_data=="<", 2, 0 )
        self.right_data = np.where(inner_data==">",4, 0)
        self.up_data = np.where(inner_data=="^",3, 0)
        self.down_data = np.where(inner_data=="v",5, 0)
        self.time =0
        self.game_status = "active"
    def check_status(self):
        data = self.get_data()
        if (self.elf.x, self.elf.y) == self.entrance:
            return 1
        if self.elf.x >= data.shape[0]-1 or self.elf.y >= data.shape[1]-1 or self.elf.x <0 or self.elf.y <0:
            return -1
        value = data[self.elf.x][self.elf.y]
        if value == -1:
            return 1
        elif value ==0:
            return 0
        else: return -1
        
    def get_data(self):
        re_inner_data = self.left_data+self.right_data+self.up_data + self.down_data
        re_inner_data = np.pad(re_inner_data,1,constant_values=0)
        return re_inner_data + self.outer_data
    def progress_env(self, n_steps, reset_state=False):
        # print("progress received n_steps ", n_steps, "reset_state ", reset_state, " time ", self.time )


        if not reset_state:
            self.time += n_steps
            roll_steps = n_steps
        else:
            roll_steps = n_steps- self.time
            self.time = n_steps
        self.left_data =np.roll(self.left_data,-roll_steps,axis =1)
        self.right_data =np.roll(self.right_data,roll_steps,axis =1)
        self.up_data =np.roll(self.up_data,-roll_steps,axis =0)
        self.down_data =np.roll(self.down_data,roll_steps,axis =0)
        # assert self.check_status() >=0, str(self.elf.x) + " " + str(self.elf.y) + " " + str(hash(str(env.get_data()))) + " time " + str(self.time)

    def backup(self):

        self.elf.backup()
        # print("call progress_env at backup")
        self.progress_env(-1)

    def step(self,direction) -> int:
        # print("call progress at step")
        self.progress_env(1)
        self.elf.step(direction)
        if self.check_status() <0:
            self.game_status = "over"
            return -1
        elif self.check_status() >0:
            self.game_status = "done"
            return 1
        else: 
            self.game_status = "active"
            return 0
    def printout(self):
        data = self.get_data()
        i=0
        for row in data:
            j=0
            print()
            for item in row:
                if (i,j) == (self.elf.x,self.elf.y):
                    if item!=0 and item != -1:
                        print("X", end="")
                    else:
                        print("E", end="")
                else:
                    if item==1:
                        print('#', end="")
                    elif item==0 or item ==-1:
                        print(".", end="")
                    elif item == 2:
                        print("<", end="")
                    elif item == 4:
                        print(">", end="")
                    elif item == 3:
                        print("^", end="")
                    elif item == 5:
                        print("v", end="")
                    else:
                        print("&", end="")
                j += 1
            i += 1

    def clone(self):
        env = copy.deepcopy(self)
        return env


In [232]:
len(state_map)

66

In [46]:
actions =['down', 'up', 'wait', 'down', 'up', 'wait', 'down', 'wait', 'right', 'right', 'wait', 'down', 'down', 'right', 'right', 'right', 'down', 'down']

env = Environment(file_path="day24/example_input.txt")
# env.printout()
print()

for action in actions:
    # print(action)
    # print(hash(str(env.get_data())))
    env.step(action)
    env.printout()
    print()
    if env.check_status() <0:
        print("invalid")
    elif env.check_status()==1:
        print("done!")

    
    



#.######
#E>&.<.#
#<..<<.#
#>&.v&.#
#>v..^<#
######.#

#E######
#.&>&..#
#.^&v^<#
#.>&.^>#
#.>..<.#
######.#

#E######
#<^<&&.#
#.&<.v.#
#><&>..#
#..><..#
######.#

#.######
#E<..&&#
#<<.<..#
#<&.>>.#
#.^&&^.#
######.#

#E######
#&.v.<>#
#<.<..<#
#.^>^&&#
#.&..&.#
######.#

#E######
#>&.<.<#
#.vv^v<#
#>..>v>#
#<....>#
######.#

#.######
#E&&^v.#
#<v.<v.#
#>>v<>.#
#>....<#
######.#

#.######
#E<>&^.#
#..<<.<#
#.&&..>#
#.&v^v.#
######.#

#.######
#<E&>>.#
#.<<.<.#
#>v>&^.#
#.v><^.#
######.#

#.######
#.&E.>&#
#<vvv^.#
#<>.>&.#
#..<>..#
######.#

#.######
#&^E^v>#
#<v<.^<#
#..&.>&#
#.<..>.#
######.#

#.######
#>>.<^<#
#.<E.<<#
#>v.><>#
#<^v^^>#
######.#

#.######
#.>&.<.#
#<..<<.#
#>&Ev&.#
#>v..^<#
######.#

#.######
#.&>&..#
#.^&v^<#
#.>&E^>#
#.>..<.#
######.#

#.######
#<^<&&.#
#.&<.v.#
#><&>E.#
#..><..#
######.#

#.######
#.<..&&#
#<<.<..#
#<&.>>E#
#.^&&^.#
######.#

#.######
#&.v.<>#
#<.<..<#
#.^>^&&#
#.&..&E#
######.#

#.######
#>&.<.<#
#.vv^v<#
#>..>v>#
#<....>#
######E#
done!


In [196]:
env.check_status()

0

((5, 6, 3006148450866359953), (24, ['down', 'up', 'wait', 'down', 'up', 'wait', 'down', 'wait', 'right', 'right', 'wait', 'down', 'down', 'right', 'up', 'up', 'wait', 'right', 'right', 'wait', 'down', 'down', 'down', 'down']))
24


[24]

In [ ]:

def fastforward(self, n_step:int):
    def move_blizzard(n_step: int, direction: str, current_position: int, field_length: int):
        #calculate the new position after moving n_step
        if direction == "right" or direction == "up": #same direction with index
            mod= (n_step + current_position)%(field_length-1)
            if mod ==0:
                return field_length-1
            else:
                return mod
        else:
            return field_length-1 - abs((current_position -n_step)%(field_length-1))
    for blizzard in self.blizzards:
        
        if blizzard.direction == "up" or blizzard.direction == "down": #vertical blizzards
            blizzard.y = move_blizzard(n_step,blizzard.direction, blizzard.y, self.max_y-self.min_y)
        else:
            blizzard.x = move_blizzard(n_step,blizzard.direction, blizzard.x, self.max_x-self.min_x)

inner_data = raw_data[1:raw_data.shape[0]-1,1:raw_data.shape[1]-1]
outer_data = np.where(raw_data=="#", 1, 0 )
outer_data[outer_data.shape[0]-1][-2] =-1
left_data = np.where(inner_data=="<", 2, 0 )
right_data = np.where(inner_data==">",4, 0)
up_data = np.where(inner_data=="^",3, 0)
down_data = np.where(inner_data=="v",5, 0)
re_inner_data = left_data+right_data+up_data + down_data
re_inner_data = np.pad(re_inner_data,1,constant_values=0)
re_data = re_inner_data + outer_data
def printout(data):
    for row in data:
        print()
        for item in row:
            if item==1:
                print('#', end="")
            elif item==0 or item ==-1:
                print(".", end="")
            elif item == 2:
                print("<", end="")
            elif item == 4:
                print(">", end="")
            elif item == 3:
                print("^", end="")
            elif item == 5:
                print("v", end="")
    
printout(re_data)

In [14]:
state_map

{(1, 1, 7217965981378803662): (21, ['wait', 'wait', 'down']),
 (0, 1, -5233929096738538225): (14, ['up']),
 (1, 1, -3011179495996423197): (20, ['wait', 'left']),
 (0, 1, -6121447156210997430): (16, ['up']),
 (0, 1, 2788479017707645568): (19, ['up']),
 (1, 1, -5233929096738538225): (17, ['wait', 'wait', 'up']),
 (0, 1, -134645775072115026): (18, ['up']),
 (1, 1, -5047791802204542517): (19, ['up']),
 (0, 1, -8690371383787918336): (18, ['up']),
 (0, 1, 3006148450866359953): (19, ['up']),
 (0, 1, -4814525041417234243): (18, ['up']),
 (2, 1, -4814525041417234243): (19, ['wait', 'up']),
 (1, 1, -4814525041417234243): (19, ['wait', 'wait', 'up']),
 (3, 1, -4814525041417234243): (9, ['wait', 'wait', 'down']),
 (2, 2, -4814525041417234243): (20, ['wait', 'wait', 'up']),
 (1, 2, -8690371383787918336): (19, ['wait', 'wait', 'up']),
 (2, 1, 3006148450866359953): (17, ['wait', 'wait', 'left']),
 (3, 1, -134645775072115026): (5, ['wait', 'wait', 'down']),
 (2, 1, -2248542712511064630): (7, ['wait', 

In [5]:
global node_count
node_count = 0
def x(node_count):
    node_count+=1
x(node_count)
node_count

0

In [203]:
# def solve(data, starting_position=(0,1), entrance=(data.shape[0]-1, data.shape[1]-2 )):
starting_position=(0,1)
entrance=(data.shape[0]-1, data.shape[1]-2 )
nodes = set([(i,j) for j in range(1, data.shape[1]-1)for i in range(1, data.shape[0]-1) ])
##adding entrance and starting nodes
nodes.add(starting_position)
nodes.add(entrance)

unvisited_nodes = nodes.copy()
distance = {node:(float("inf"),False,[],0) for node in nodes}
source = (0,1)
MAX_WAIT = 1

def get_min_distance_node(distance):
    y = [item for item in distance.items() if item[1][1] is False]
    if len(y)==0: return None
    y.sort(key= lambda x: x[1][0])
    return y[0][0]
def test_action(action, env):

        reward = env.step(action)
        x,y, time = env.elf.x, env.elf.y, env.time
        env.backup()

        return reward >= 0, (x,y, time)
def get_adjacents(node, env,unvisited_nodes):
    output=[]
    ok_actions =[]
    time_before_action = env.time
    for action in ["wait","up","down", "left", "right"]:
        result = test_action(action, env)
        if result[0]:
            if (result[1][0],result[1][1])  in unvisited_nodes:
                output.append(((result[1][0],result[1][1],result[1][2]),[action],1))
                ok_actions.append(action)
    waits=[]
    wait_count =0
    actions_to_wait_for = set(["up","down", "left", "right","wait"]) - set(ok_actions) 

    if len(actions_to_wait_for) >0:
        for _ in range(MAX_WAIT):
            waits.append("wait")
            wait_count +=1
            env.step("wait")
            for action in actions_to_wait_for:
                result = test_action(action, env)
                if result[0]:
                    if (result[1][0],result[1][1])  in unvisited_nodes:
                        output.append(((result[1][0],result[1][1],result[1][2]),waits+[action],wait_count+1))
        env.progress_env(-wait_count) #reverse environment back
    time_after_action = env.time

    assert time_after_action == time_before_action, str(time_before_action) + "   " + str(time_after_action)
    return output

#initialize distance list
distance[source]=(0,False,[],0)
while len(unvisited_nodes) >0:
    path_node = get_min_distance_node(distance)

    old_distance_val = distance.get(path_node)
    distance[path_node] = (old_distance_val[0], True, old_distance_val[2], old_distance_val[3]) #adding to path
    unvisited_nodes.remove(path_node) #remove from unvisited node
    env.elf.x, env.elf.y = path_node[0],path_node[1] #update elf position for environment

    path_time = distance[path_node][0]
    if path_time == float("inf"):
        print("invalid path node exitting")
        print("path node ", path_node)
        break
    env.progress_env(path_time,reset_state=True) #update the environment progress
    assert env.time == path_time

    adjacent_nodes = get_adjacents(path_node,env,unvisited_nodes)

    for node in adjacent_nodes:
        coordinates = (node[0][0], node[0][1])
        new_distance = node[2] + path_time #update distance
        actions = old_distance_val[2]+node[1]  #update actions, appending path node's actions with new actions needed to reach adjacent node
        node_time = node[0][2]  #update time
        assert new_distance == len(actions), "new distance "+ new_distance + " len(actions) " + len(actions)
        distance[coordinates] = (new_distance, False, actions, node_time) #adding to path


invalid path node exitting
path node  (2, 5)


In [204]:
distance[(1, 2)]

(8, True, ['down', 'down', 'wait', 'right', 'right', 'up', 'wait', 'left'], 8)

In [205]:
actions = distance[(1, 2)][2]
env = Environment(file_path="day24/example_input.txt")
env.printout()

for action in actions:
    print()

    print(action)
    env.step(action)
    env.printout()



#.######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#
down

#.######
#.>&.<.#
#<..<<.#
#>&.v&.#
#>v..^<#
######.#
down

#.######
#.&>&..#
#.^&v^<#
#.>&.^>#
#.>..<.#
######.#
wait

#.######
#<^<&&.#
#.&<.v.#
#><&>..#
#..><..#
######.#
right

#.######
#.<..&&#
#<<.<..#
#<&.>>.#
#.^&&^.#
######.#
right

#.######
#&.v.<>#
#<.<..<#
#.^>^&&#
#.&..&.#
######.#
up

#.######
#>&.<.<#
#.vv^v<#
#>..>v>#
#<....>#
######.#
wait

#.######
#.&&^v.#
#<v.<v.#
#>>v<>.#
#>....<#
######.#
left

#.######
#.<>&^.#
#..<<.<#
#.&&..>#
#.&v^v.#
######.#

In [165]:
actions[:20]

['down',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait']

In [151]:
env.elf.x

1

In [152]:
env.elf.y

2